# 1. Data reading and preprocessing
 

## 1.1 imports

In [2]:
import json
import convokit
from convokit.politenessStrategies.politenessStrategies import PolitenessStrategies

import nltk
import pandas as pd
!python -m spacy download en
from convokit import Corpus, download
import pickle
import sklearn
#from convokit.classifier.classifier import Classifier
from convokit import TextParser
from nltk.tokenize.punkt import PunktSentenceTokenizer
from spacy.lang.en import English
!pip3 install -U scikit-learn==0.19.2
import politeness
import joblib

/home/cormak/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, defaultdict
Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/cormak/.local/lib/python3.8/site-packages/en_core_web_sm -->
/home/cormak/.local/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn==0.19.2 in /home/cormak/.local/lib/python3.8/site-packages (0.19.2)


## 1.2 politeness

* I decided to take the annotated wiki corpus to train our politeness classifier as I could notfind a working pre trained model

In [3]:
parser = TextParser(verbosity=1000)
train_corpus = Corpus(filename=download('wiki-politeness-annotated'))
#parser.transform(train_corpus)
ps = PolitenessStrategies()
#ps.transform(train_corpus)
#clf = Classifier(obj_type='utterance', pred_feats=['politeness_strategies'], 
#                 labeller=lambda utt: utt.meta['Binary']==1)
#clf.fit(train_corpus)

Dataset already exists at /home/cormak/.convokit/downloads/wiki-politeness-annotated


In [4]:
deception = Corpus(filename=download("diplomacy-corpus"))
deception=parser.transform(deception)
deception=ps.transform(deception)


Dataset already exists at /home/cormak/.convokit/downloads/diplomacy-corpus
1000/17289 utterances processed
2000/17289 utterances processed
3000/17289 utterances processed
4000/17289 utterances processed
5000/17289 utterances processed
6000/17289 utterances processed
7000/17289 utterances processed
8000/17289 utterances processed
9000/17289 utterances processed
10000/17289 utterances processed
11000/17289 utterances processed
12000/17289 utterances processed
13000/17289 utterances processed
14000/17289 utterances processed
15000/17289 utterances processed
16000/17289 utterances processed
17000/17289 utterances processed
17289/17289 utterances processed


* creation of features for politeness

In [7]:
import numpy as np
from scipy.sparse import csr_matrix

import _pickle
nltk.download('punkt')
MODEL_FILENAME='./politeness/politeness-svm.p'
clf = _pickle.load(open(MODEL_FILENAME, 'rb'), encoding='latin1', fix_imports=True)


def score(utterance):
    # Vectorizer returns {feature-name: value} dict
    features = utterance.meta['politeness_strategies']
    print(features)
    fv = [features[f] for f in sorted(features.keys())]
    # Single-row sparse matrix
    X = csr_matrix(np.asarray([fv]))
    probs = clf.predict_proba(X)
    # Massage return format
    probs = {"polite": probs[0][1], "impolite": probs[0][0]}
    return probs

probs = score(deception.random_utterance())


[nltk_data] Downloading package punkt to /home/cormak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
{'feature_politeness_==Please==': 0, 'feature_politeness_==Please_start==': 0, 'feature_politeness_==HASHEDGE==': 1, 'feature_politeness_==Indirect_(btw)==': 0, 'feature_politeness_==Hedges==': 1, 'feature_politeness_==Factuality==': 0, 'feature_politeness_==Deference==': 0, 'feature_politeness_==Gratitude==': 0, 'feature_politeness_==Apologizing==': 0, 'feature_politeness_==1st_person_pl.==': 0, 'feature_politeness_==1st_person==': 0, 'feature_politeness_==1st_person_start==': 1, 'feature_politeness_==2nd_person==': 1, 'feature_politeness_==2nd_person_start==': 0, 'feature_politeness_==Indirect_(greeting)==': 0, 'feature_politeness_==Direct_question==': 0, 'feature_politeness_==Direct_start==': 0, 'feature_politeness_==HASPOSITIVE==': 0, 'feature_politeness_==HASNEGATIVE==': 0, 'feature_politeness_==SUBJUNCTIVE==': 0, 'feature_politeness_==INDICATIVE==': 0}


ValueError: X.shape[1] = 21 should be equal to 1447, the number of features at training time

In [9]:
import scipy
import numpy as np
packages2versions = [("scikit-learn", sklearn, "0.18.1"),
                     ("numpy", np, "1.12.0"),
                     ("nltk", nltk, "3.2.1"),
                     ("scipy", scipy, "0.18.1")]

for name, package, expected_v in packages2versions:
    if package.__version__ < expected_v:
        sys.stderr.write("Warning: package '%s', expected version >= %s, detec",
                         "ted %s. Code functionality not guaranteed.\n"
                         % (name, expected_v, package.__version__))

from politeness.features.vectorizer import PolitenessFeatureVectorizer
vectorizer = PolitenessFeatureVectorizer()

import _pickle
nltk.download('punkt')

def score(request):
    """
    :param request - The request document to score
    :type request - dict with 'sentences' and 'parses' field
        sample (taken from test_documents.py)--
        {
            'sentences': [
                "Have you found the answer for your question?",
                "If yes would you please share it?"
            ],
            'parses': [
                ["csubj(found-3, Have-1)", "dobj(Have-1, you-2)",
                 "root(ROOT-0, found-3)", "det(answer-5, the-4)",
                 "dobj(found-3, answer-5)", "poss(question-8, your-7)",
                 "prep_for(found-3, question-8)"],
                ["prep_if(would-3, yes-2)", "root(ROOT-0, would-3)",
                 "nsubj(would-3, you-4)", "ccomp(would-3, please-5)",
                 "nsubj(it-7, share-6)", "xcomp(please-5, it-7)"]
            ]
        }

    returns class probabilities as a dict
        { 'polite': float, 'impolite': float }
    """
    # Vectorizer returns {feature-name: value} dict
    print(request)
    features = vectorizer.features(request)
    print(features)
    fv = [features[f] for f in sorted(features.keys())]
    print(fv)
    # Single-row sparse matrix
    X = csr_matrix(np.asarray([fv]))
    probs = clf.predict_proba(X)
    # Massage return format
    probs = {"polite": probs[0][1], "impolite": probs[0][0]}
    return probs


score({
            'sentences': [
                "Have you found the answer for your question?",
                "If yes would you please share it?"
            ],
            'parses': [
                ["csubj(found-3, Have-1)", "dobj(Have-1, you-2)",
                 "root(ROOT-0, found-3)", "det(answer-5, the-4)",
                 "dobj(found-3, answer-5)", "poss(question-8, your-7)",
                 "prep_for(found-3, question-8)"],
                ["prep_if(would-3, yes-2)", "root(ROOT-0, would-3)",
                 "nsubj(would-3, you-4)", "ccomp(would-3, please-5)",
                 "nsubj(it-7, share-6)", "xcomp(please-5, it-7)"]
            ]
        })

[nltk_data] Downloading package punkt to /home/cormak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
{'sentences': ['Have you found the answer for your question?', 'If yes would you please share it?'], 'parses': [['csubj(found-3, Have-1)', 'dobj(Have-1, you-2)', 'root(ROOT-0, found-3)', 'det(answer-5, the-4)', 'dobj(found-3, answer-5)', 'poss(question-8, your-7)', 'prep_for(found-3, question-8)'], ['prep_if(would-3, yes-2)', 'root(ROOT-0, would-3)', 'nsubj(would-3, you-4)', 'ccomp(would-3, please-5)', 'nsubj(it-7, share-6)', 'xcomp(please-5, it-7)']]}
{'feature_politeness_==Please==': 1, 'feature_politeness_==Please_start==': 0, 'feature_politeness_==Indirect_(btw)==': 0, 'feature_politeness_==Hedges==': 0, 'feature_politeness_==Factuality==': 0, 'feature_politeness_==Deference==': 0, 'feature_politeness_==Gratitude==': 0, 'feature_politeness_==Apologizing==': 0, 'feature_politeness_==1st_person_pl.==': 0, 'feature_politeness_==1st_person==': 0, 'feature_politeness_==1

ValueError: X.shape[1] = 21 should be equal to 1447, the number of features at training time

In [10]:

sentence=parser.transform_utterance("Have you found the answer for your question?")
sentence=ps.transform_utterance(sentence)

clf.transform_objs([sentence])[0].meta

AttributeError: 'SVC' object has no attribute 'transform_objs'

In [75]:
{
    'sentences': [
        "Have you found the answer for your question?",
        "If yes would you please share it?"
    ],
    'parses': [
        ["csubj(found-3, Have-1)", "dobj(Have-1, you-2)",
            "root(ROOT-0, found-3)", "det(answer-5, the-4)",
            "dobj(found-3, answer-5)", "poss(question-8, your-7)",
            "prep_for(found-3, question-8)"],
        ["prep_if(would-3, yes-2)", "root(ROOT-0, would-3)",
            "nsubj(would-3, you-4)", "ccomp(would-3, please-5)",
            "nsubj(it-7, share-6)", "xcomp(please-5, it-7)"]
    ]
}
deception = parser.transform(deception)
deception = ps.transform(deception, markers=True)
summary = ps.summarize(deception)


1000/17289 utterances processed
2000/17289 utterances processed
3000/17289 utterances processed
4000/17289 utterances processed
5000/17289 utterances processed
6000/17289 utterances processed
7000/17289 utterances processed
8000/17289 utterances processed
9000/17289 utterances processed
10000/17289 utterances processed
11000/17289 utterances processed
12000/17289 utterances processed
13000/17289 utterances processed
14000/17289 utterances processed
15000/17289 utterances processed
16000/17289 utterances processed
17000/17289 utterances processed
17289/17289 utterances processed


In [74]:
parser.transform_utterance("Have you found the answer for your question?")

Utterance({'obj_type': 'utterance', 'meta': {'parsed': [{'rt': 2, 'toks': [{'tok': 'Have', 'tag': 'VBP', 'dep': 'aux', 'up': 2, 'dn': []}, {'tok': 'you', 'tag': 'PRP', 'dep': 'nsubj', 'up': 2, 'dn': []}, {'tok': 'found', 'tag': 'VBN', 'dep': 'ROOT', 'dn': [0, 1, 4, 8]}, {'tok': 'the', 'tag': 'DT', 'dep': 'det', 'up': 4, 'dn': []}, {'tok': 'answer', 'tag': 'NN', 'dep': 'dobj', 'up': 2, 'dn': [3, 5]}, {'tok': 'for', 'tag': 'IN', 'dep': 'prep', 'up': 4, 'dn': [7]}, {'tok': 'your', 'tag': 'PRP$', 'dep': 'poss', 'up': 7, 'dn': []}, {'tok': 'question', 'tag': 'NN', 'dep': 'pobj', 'up': 5, 'dn': [6]}, {'tok': '?', 'tag': '.', 'dep': 'punct', 'up': 2, 'dn': []}]}]}, 'vectors': [], 'speaker': None, 'conversation_id': None, 'reply_to': None, 'timestamp': None, 'text': 'Have you found the answer for your question?', 'owner': None, 'id': None})

In [121]:

# test stanza
from stanza.server import CoreNLPClient
import stanza as stanza
stanza.install_corenlp()
client = CoreNLPClient(
        annotators=['tokenize','ssplit','pos', 'lemma','ner',  'depparse','coref', 'kbp','tokenize','ssplit','pos'],
        timeout=30000,
        memory='16G', be_quiet=True)



2020-12-04 23:25:44 WARNING: Directory /home/cormak/stanza_corenlp already exists. Please install CoreNLP to a new directory.
2020-12-04 23:25:44 INFO: Writing properties to tmp file: corenlp_server-a16f973794894b5b.props


In [118]:
ann = client.annotate(
"Have you found the answer for your question? If yes would you please share it?"
)
sentence = ann.sentence[0]
sentence.parseTree

2020-12-04 23:23:39 INFO: Starting server with command: java -Xmx16G -cp /home/cormak/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-a6babde155a04ceb.props -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref -preload -outputFormat serialized


child {
  child {
    child {
      value: "Have"
    }
    value: "VBP"
    score: -0.62845379114151
  }
  child {
    child {
      child {
        value: "you"
      }
      value: "PRP"
      score: -2.1667263507843018
    }
    value: "NP"
    score: -2.8400943279266357
  }
  child {
    child {
      child {
        value: "found"
      }
      value: "VBN"
      score: -4.834004878997803
    }
    child {
      child {
        child {
          child {
            value: "the"
          }
          value: "DT"
          score: -0.5893322825431824
        }
        child {
          child {
            value: "answer"
          }
          value: "NN"
          score: -8.24889850616455
        }
        value: "NP"
        score: -10.436527252197266
      }
      child {
        child {
          child {
            value: "for"
          }
          value: "IN"
          score: -2.6398026943206787
        }
        child {
          child {
            child {
              valu

In [120]:
print(sentence.basicDependencies)


node {
  sentenceIndex: 0
  index: 1
}
node {
  sentenceIndex: 0
  index: 2
}
node {
  sentenceIndex: 0
  index: 3
}
node {
  sentenceIndex: 0
  index: 4
}
node {
  sentenceIndex: 0
  index: 5
}
node {
  sentenceIndex: 0
  index: 6
}
node {
  sentenceIndex: 0
  index: 7
}
node {
  sentenceIndex: 0
  index: 8
}
node {
  sentenceIndex: 0
  index: 9
}
edge {
  source: 3
  target: 1
  dep: "aux"
  isExtra: false
  sourceCopy: 0
  targetCopy: 0
  language: UniversalEnglish
}
edge {
  source: 3
  target: 2
  dep: "nsubj"
  isExtra: false
  sourceCopy: 0
  targetCopy: 0
  language: UniversalEnglish
}
edge {
  source: 3
  target: 5
  dep: "obj"
  isExtra: false
  sourceCopy: 0
  targetCopy: 0
  language: UniversalEnglish
}
edge {
  source: 3
  target: 9
  dep: "punct"
  isExtra: false
  sourceCopy: 0
  targetCopy: 0
  language: UniversalEnglish
}
edge {
  source: 5
  target: 4
  dep: "det"
  isExtra: false
  sourceCopy: 0
  targetCopy: 0
  language: UniversalEnglish
}
edge {
  source: 5
  targ